In [22]:
import os
import dask.dataframe as dd
from dask.distributed import Client
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)

import sys
sys.path.insert(0,'/home/djl34/lab_pd/bin')
import genomic


pd_data_dir = "/home/djl34/lab_pd/data"
aso_data_dir = "/home/djl34/lab_pd/aso/data"
KL_data_dir = "/home/djl34/lab_pd/kl/data"
scratch_dir = "/n/scratch3/users/d/djl34"

In [23]:
names_list = ["Chrom", "start", "end", "name", "LaBranchoR_score", "strand"]

df = pd.read_csv(aso_data_dir + "/LaBranchoR/lstm.gencode_v19.hg19.top.bed", sep = "\t", names = names_list)

In [24]:
df = df[df["Chrom"] == "chr19"]

In [25]:
def get_range(start, end):
    return range(start + 1, end + 1)

In [26]:
df["hg38_start"] = df.apply(lambda row: genomic.get_hg38_pos(row["Chrom"], row["start"]), axis=1)
df["hg38_end"] = df.apply(lambda row: genomic.get_hg38_pos(row["Chrom"], row["end"]), axis=1)

In [28]:
df[df["hg38_start"].isna()]

,Chrom,start,end,name,LaBranchoR_score,strand,hg38_start,hg38_end
102764,chr19,40408876,40408877,40408847,0.861932,-,None,None


In [27]:
df["hg38_start"] = df["hg38_start"].astype(int)
df["hg38_end"] = df["hg38_end"].astype(int)

TypeError: int() argument must be a string, a bytes-like object or a number, not 'NoneType'

In [17]:
df['hg38_min'] = df[['hg38_start','hg38_end']].min(axis=1)
df['hg38_max'] = df[['hg38_start','hg38_end']].max(axis=1)


In [18]:
df["Pos"] = df.apply(lambda row: get_range(row["hg38_min"],row["hg38_max"]), axis=1)

In [19]:
df

,Chrom,start,end,name,LaBranchoR_score,strand,hg38_start,hg38_end,Pos,hg38_min,hg38_max
130262,chr22,16256702,16256703,16256676,0.804874,-,15721261,15721260,(15721261),15721260,15721261
130263,chr22,16258325,16258326,16258302,0.910530,-,15719638,15719637,(15719638),15719637,15719638
130264,chr22,16267115,16267116,16267094,0.573785,-,15710848,15710847,(15710848),15710847,15710848
130265,chr22,16268203,16268204,16268180,0.902241,-,15709760,15709759,(15709760),15709759,15709760
130266,chr22,16269967,16269968,16269942,0.885595,-,15707996,15707995,(15707996),15707995,15707996
...,...,...,...,...,...,...,...,...,...,...,...
134688,chr22,51216433,51216434,51216408,0.946459,-,50778005,50778006,(50778006),50778005,50778006
134689,chr22,51220805,51220806,51220774,0.864709,-,50782377,50782378,(50782378),50782377,50782378
134690,chr22,51220805,51220806,51220778,0.892884,-,50782377,50782378,(50782378),50782377,50782378
134691,chr22,51221500,51221501,51221472,0.686466,-,50783072,50783073,(50783073),50783072,50783073


In [20]:
df["Pos"]

130262    (15721261)
130263    (15719638)
130264    (15710848)
130265    (15709760)
130266    (15707996)
             ...    
134688    (50778006)
134689    (50782378)
134690    (50782378)
134691    (50783073)
134692    (50783311)
Name: Pos, Length: 4431, dtype: object

In [21]:
df.explode('Pos')

,Chrom,start,end,name,LaBranchoR_score,strand,hg38_start,hg38_end,Pos,hg38_min,hg38_max
130262,chr22,16256702,16256703,16256676,0.804874,-,15721261,15721260,15721261,15721260,15721261
130263,chr22,16258325,16258326,16258302,0.910530,-,15719638,15719637,15719638,15719637,15719638
130264,chr22,16267115,16267116,16267094,0.573785,-,15710848,15710847,15710848,15710847,15710848
130265,chr22,16268203,16268204,16268180,0.902241,-,15709760,15709759,15709760,15709759,15709760
130266,chr22,16269967,16269968,16269942,0.885595,-,15707996,15707995,15707996,15707995,15707996
...,...,...,...,...,...,...,...,...,...,...,...
134688,chr22,51216433,51216434,51216408,0.946459,-,50778005,50778006,50778006,50778005,50778006
134689,chr22,51220805,51220806,51220774,0.864709,-,50782377,50782378,50782378,50782377,50782378
134690,chr22,51220805,51220806,51220778,0.892884,-,50782377,50782378,50782378,50782377,50782378
134691,chr22,51221500,51221501,51221472,0.686466,-,50783072,50783073,50783073,50783072,50783073


NameError: name 'get_range' is not defined